In [1]:
import numpy as np
import tensorflow as tf

In [2]:
data_dict = np.load("vgg16.npy",encoding='latin1').item()

FileNotFoundError: [Errno 2] No such file or directory: 'vgg16.npy'

In [ ]:
data_dict.keys()

In [ ]:
data_dict['conv1_1'][0]

In [ ]:
data_dict['conv1_1'][1].shape

In [ ]:
v = tf.truncated_normal([2,2],0,1)

In [ ]:
with tf.Session() as sess:
    print(sess.run(v))

In [3]:
import os
import cv2

In [4]:
def get_images_labels_list():
    images_list=[]
    labels_list=[]
    with open("/home/linx-dl/hzr/2012_train.txt",'r') as f:
        for line in f:
            image_full_name=line.split('\n')[0]
           # print(image_full_name)
            images_list.append(image_full_name)
            label_name = line.split('/')[-1].split('.')[0]+'.txt'
            label_full_name=os.path.join("/home/linx-dl/hzr/VOCdevkit/VOC2012/labels",label_name)
           
          #  print(label_full_name)
            labels_list.append(label_full_name)
    return images_list,labels_list

In [5]:
images_list,labels_list  = get_images_labels_list()
       #self.labels_list = self.get_labels_list()

In [6]:
def parse_function(filename, label):
    image_string = tf.read_file(filename)
    
    label = tf.read_file(label)
    #label = tf.decode_raw(label,tf.uint8)
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_images(image, [416, 416])
    return image, label

In [7]:
def _read_py_function(filename,label):
    image_decoded = cv2.imread(filename.decode(), cv2.IMREAD_GRAYSCALE)
    return image_decoded, label

In [8]:
def _resize_function(image_decoded, label):
    image_decoded.set_shape([None, None, None])
    image_resized = tf.image.resize_images(image_decoded, [28, 28])
    return image_resized, label

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((tf.constant(images_list),tf.constant(labels_list)))
#dataset = dataset.map(lambda filename, label: tuple(tf.py_func(_read_py_function, [filename, label], [tf.uint8, tf.string])),num_parallel_calls=4)
dataset = dataset.map(parse_function,num_parallel_calls=4)
dataset = dataset.shuffle(len(images_list))
dataset = dataset.batch(10)
dataset = dataset.prefetch(1)

iterator = dataset.make_initializable_iterator()


In [10]:
import time

In [11]:
with tf.Session() as sess:
    sess.run(iterator.initializer)
   
    batch_images,batch_labels = sess.run(iterator.get_next())

In [12]:
image_list=[]
for i,label in enumerate(batch_labels):
    label_list=[]
    for j,v in enumerate(label.decode().splitlines()):
        label_list.append([float(vv)for vv in v.split(' ')])
    image_list.append(label_list)

In [13]:
image_list

[[[18.0, 0.382, 0.48930481283422456, 0.764, 0.7486631016042781]],
 [[9.0, 0.5351473922902494, 0.48, 0.7936507936507937, 0.844]],
 [[17.0, 0.588, 0.644, 0.8186666666666667, 0.672],
  [14.0, 0.852, 0.657, 0.2906666666666667, 0.642],
  [14.0, 0.588, 0.608, 0.568, 0.78]],
 [[7.0, 0.4425, 0.47000000000000003, 0.655, 0.7666666666666667]],
 [[14.0, 0.68, 0.512, 0.636, 0.9706666666666667],
  [11.0, 0.201, 0.7653333333333333, 0.122, 0.3893333333333333],
  [6.0, 0.41000000000000003, 0.41466666666666663, 0.1, 0.09333333333333332],
  [6.0, 0.872, 0.36, 0.08, 0.08],
  [14.0, 0.107, 0.5093333333333333, 0.20600000000000002, 0.9226666666666666],
  [14.0, 0.11, 0.3253333333333333, 0.20400000000000001, 0.21333333333333332]],
 [[9.0, 0.629, 0.37066666666666664, 0.106, 0.08533333333333333],
  [9.0, 0.404, 0.46266666666666667, 0.22, 0.16799999999999998],
  [9.0, 0.322, 0.5453333333333333, 0.184, 0.21066666666666667]],
 [[10.0, 0.405, 0.7749244712990937, 0.806, 0.44410876132930516],
  [8.0, 0.93900000000000

In [14]:
def iou_wh(r1, r2):

    min_w = min(r1[0],r2[0])
    min_h = min(r1[1],r2[1])
    area_r1 = r1[0]*r1[1]
    area_r2 = r2[0]*r2[1]

    intersect = min_w * min_h
    union = area_r1 + area_r2 - intersect

    return intersect/union

def get_active_anchors(roi, anchors):
    indxs = []
    iou_max, index_max = 0, 0
    
    for i,a in enumerate(anchors):
        iou = iou_wh(roi[1:], a)
        if iou>0.7:
            indxs.append(i)
        if iou > iou_max:
            iou_max, index_max = iou, i

    if len(indxs) == 0:
        indxs.append(index_max)

    return indxs

def get_grid_cell(roi):

    x_center = roi[1] + roi[3]/2.0
    y_center = roi[2] + roi[4]/2.0

    grid_x = int(x_center/float(1/13))
    grid_y = int(y_center/float(1/13))

    return grid_x, grid_y
def roi2label(roi, anchor):

    x_center = roi[1]+roi[3]/2.0
    y_center = roi[2]+roi[4]/2.0

    grid_x = x_center/float(1/13)
    grid_y = y_center/float(1/13)
    
    grid_x_offset = grid_x - int(grid_x)
    grid_y_offset = grid_y - int(grid_y)

    roi_w_scale = roi[3]/(1/anchor[0])
    roi_h_scale = roi[4]/(1/anchor[1])

    label=[grid_x_offset, grid_y_offset, roi_w_scale, roi_h_scale]
    
    return label

In [30]:
label = np.zeros([10,13, 13, 5, 6], dtype=np.float32)
anchors=[1.3221, 1.73145, 3.19275, 4.00944, 5.05587, 8.09892, 9.47112, 4.84053, 11.2364, 10.0071]
anchor_list=[]
for i in range(0,len(anchors),2):
    anchor_list.append([anchors[i]/13,anchors[i+1]/13])
for image_index in range(len(image_list)):#every image
    for label_index in range(len(image_list[image_index])):#every label
       # print(image_list[image_index][label_index])
        activate_anchor_index=get_active_anchors(image_list[image_index][label_index],anchor_list)
        
        grid_x,grid_y = get_grid_cell(image_list[image_index][label_index])
        for index in activate_anchor_index:
            anchor_label= roi2label(image_list[image_index][label_index],anchor_list[index])
            label[image_index,grid_x,grid_y,index] = np.concatenate((anchor_label, [int(image_list[image_index][label_index][0])], [1.0]))

In [31]:
label[0,9,...]

array([[[  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
      

In [ ]:
image_list=[]
for l in batch_labels:
    label_list=[]
    for j in l.decode().splitlines():
        print([float(value) for value in j.split(' ')])
    images_list.append(label_list)

In [ ]:
image_list

In [ ]:
batch_images[0,:]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [ ]:
plt.imshow(batch_images[0,:])

In [1]:
import numpy as np

In [3]:
np.array([1])

array([1])

In [4]:
[1]

[1]